# two-way ANOVA
- 집단을 구분하는 변수(=요인)이 2개일 때, 요인 간 상호작용을 파악하기 위함. (요인의 개수가 2개(=열이 2개)면 two-way anova인듯?)
- 요인이 n개면 n-way ANOVA

# one-way ANOVA 비교
- 상호작용 효과를 분석하는게 중요
- 상호작용 효과가 유의미한 경우(p-value < 0.05) 단순효과분석 결과를 상세하게 첨부

# 주요 용어
- 주 효과: 다른 요인(집단구분 변수)과 상관없이, 한 요인의 수준(집단)에 따라 효과가 유의미하게 달라질 때, '주효과'가 있다고 함
- 상호작용 효과: 한 요인의 수준에 따른 효과의 차이가 또 다른 요인의 수준에 따라달라질 때, '요인들 간 상호작용(효과)'이 존재한다고 함

# 균형설계 vs 비균형설계

- 균형설계: 각 집단/조건별 표본수가 동일한 경우
- 비균형설계: 각 집단/조건별 표본수가 동일하지 않은 경우
- 균형설계와 비균형설계에 따라 계산 방법이 다름

## 데이터 준비

- 독립변수 - poison, treat
- 종속변수 - time (연속형)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./poison.csv", index_col=0)
print(df.head(), end='\n\n')
print("poison의 레벨: {}\ntreat의 레벨: {}".format(df.poison.unique(), df.treat.unique()))

   time  poison treat
1  0.31       1     A
2  0.45       1     A
3  0.46       1     A
4  0.43       1     A
5  0.36       2     A

poison의 레벨: [1 2 3]
treat의 레벨: ['A' 'B' 'C' 'D']


## 균형설계 확인
- 요인1(poison)의 샘플 수가 모두 동일한가?
- 요인2(treat)의 샘플 수가 모두 동일한가?
- 요인1과 요인2에 대해서 종속변수의 샘플수가 모두 동일한가?

In [3]:
# poision 요인으로 구분한 집단별 표본 수가 모두 동일한가?
df.groupby('poison').agg(len)

,time,treat
poison,,
1,16,16
2,16,16
3,16,16


In [4]:
# treat 요인으로 구분한 집단별 표본 수가 모두 동일한가?
df.groupby('treat').agg(len)

,time,poison
treat,,
A,12,12
B,12,12
C,12,12
D,12,12


In [5]:
# poison, treat 요인으로 구분한 각 집단별 표본수가 모두 동일한가?
df.groupby(['treat', "poison"]).agg(len)

time
treat poison      
A     1          4
      2          4
      3          4
B     1          4
      2          4
      3          4
C     1          4
      2          4
      3          4
D     1          4
      2          4
      3          4

# two-way anova 실시

In [6]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [7]:
model = ols('time ~ C(poison) * C(treat)', df).fit()
anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
C(poison),2.0,1.033012,0.516506,23.221737,3.331440e-07
C(treat),3.0,0.921206,0.307069,13.805582,3.777331e-06
C(poison):C(treat),6.0,0.250138,0.041690,1.874333,1.122506e-01
Residual,36.0,0.800725,0.022242,NaN,NaN


# 결과 해석

- poison: F(2, 36) = 23.222, p < 0.05로 유의미. 즉 poison의 수준에 따라 평균에 차이가 난다고 볼 수 있음
- treat: F(3, 36) = 13.806, p < 0.05로 유의미. 즉 treat의 수준에 따라 평균에 차이가 난다고 볼 수 있음
- poison:treat: F(6, 36) = 1.874, p > 0.05로 유의미하지 않음. 상호작용 효과는 발견하지 못함


# 결과 보고 예시 - 상호작용이 유의미하지 않은 경우

time에 대하여 poison과 treat를 요인으로 하는 이원분산분석을 실시한 결과,

poison의 주효과는 유의미하였으며(F(2, 36) = 23.222, p < 0.05),

treat의 주효과 또한 유의미하였으나(F(3, 36) = 13.806, p < 0.05), 

poison과 treat의 유의미한 상호작용효과는 발견할 수 없었다(F(6, 36) = 1.874, p > 0.05).


---

# 비균형자료의 이원분산분석

In [34]:
import pandas as pd
dat = pd.read_csv('./poison.csv', index_col=0)

In [39]:
dat2 = dat.loc[1:,:]  # 데이터의 첫 행을 제거하여 비균형으로 만듬. 집단 별 표본수가 동일하지 않으므로 비균형설계자료

In [37]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

model = ols('time ~ C(poison) * C(treat)', dat2).fit()
anova_lm(model, typ=3)

,sum_sq,df,F,PR(>F)
Intercept,0.680625,1.0,30.600393,0.000003
C(poison),0.082217,2.0,1.848200,0.172157
C(treat),0.453950,3.0,6.803085,0.000947
C(poison):C(treat),0.250137,6.0,1.874333,0.112251
Residual,0.800725,36.0,NaN,NaN


In [42]:
# 변수를 바꿔도 결과는 동일
model = ols('time ~ C(treat) * C(poison)', dat2).fit()
anova_lm(model, typ=3)

,sum_sq,df,F,PR(>F)
Intercept,0.680625,1.0,30.600393,0.000003
C(treat),0.453950,3.0,6.803085,0.000947
C(poison),0.082217,2.0,1.848200,0.172157
C(treat):C(poison),0.250137,6.0,1.874333,0.112251
Residual,0.800725,36.0,NaN,NaN


# 결과 해석
- treat만 p < 0.05 이므로 통계적으로 유의미하다

# 결과 보고 예시 - 상호작용효과가 유의미한 경우

poison과 treat의 유의미한 상호작용 효과에 대하여 단순효과분석을 실시한 결과,

poison이 1인 집단에서는 treat의 단순 주효과는 유의미하였고(F(3,36) = 6.8031, p < 0.05), 

poison이 2인 집단도 treat의 단순 주효과는 유의미하였으나(F(3,36) = 10.0367, p < 0.05), 

poison이 3인 집단에서는 treat의 단순 주효과가 유의미하지 않았다(F(3,36) = 0.7145, p > 0.05).